In [1]:
import fitsio
import numpy as np
import healpy as hp
import pandas as pd

# For the moment I'm using Pandas (based on numpy), which is a very useful tool for
# handling tables and data.
# I'll properly put them into classes or more organized later

In [2]:
#12GB file, be carefull

file_name = 'spAll-v5_10_0.fits'

In [3]:
#The resolution of the map

NSide = 2**5

In [20]:
# Columns we are interested in

fits = fitsio.FITS(file_name) #, iter_row_buffer=2000)
cols = ['RA','DEC','THING_ID','MJD','PLATE','FIBERID','BOSS_TARGET1','EBOSS_TARGET0','EBOSS_TARGET1', 'ZWARNING']

In [21]:
# to show all names in file

#fits[1]

In [22]:
%%time

# Reading columns
#http://stackoverflow.com/questions/30283836/creating-pandas-dataframe-from-numpy-array-leads-to-strange-errors
d = {col: fits[1][col].read().byteswap().newbyteorder() for col in cols}

CPU times: user 26 s, sys: 1min 34s, total: 2min
Wall time: 3min 48s


In [56]:
df = pd.DataFrame(d)
print 'number of rows =', len(df)
df.head()

number of rows = 3008000


,BOSS_TARGET1,DEC,EBOSS_TARGET0,EBOSS_TARGET1,FIBERID,MJD,PLATE,RA,THING_ID,ZWARNING
0,0,26.188042,0,2,1,57346,10000,30.012094,339167825,646
1,0,26.582945,0,1024,2,57346,10000,31.282286,342410358,0
2,0,26.608501,0,2,3,57346,10000,31.279594,342410449,4
3,0,26.682522,0,2048,4,57346,10000,31.237546,342675982,0
4,0,26.866054,0,2,5,57346,10000,31.394521,344112497,4


## Computing pixels

In [57]:
'''map celestial coordinates to angular'''
phi_rad   = lambda ra : ra*np.pi/180.
theta_rad = lambda dec: (90.0 - dec)*np.pi/180.

df['PIX'] = hp.ang2pix(NSide, theta_rad(df['DEC']), phi_rad(df['RA']))

In [58]:
df.head()

,BOSS_TARGET1,DEC,EBOSS_TARGET0,EBOSS_TARGET1,FIBERID,MJD,PLATE,RA,THING_ID,ZWARNING,PIX
0,0,26.188042,0,2,1,57346,10000,30.012094,339167825,646,3403
1,0,26.582945,0,1024,2,57346,10000,31.282286,342410358,0,3403
2,0,26.608501,0,2,3,57346,10000,31.279594,342410449,4,3403
3,0,26.682522,0,2048,4,57346,10000,31.237546,342675982,0,3403
4,0,26.866054,0,2,5,57346,10000,31.394521,344112497,4,3275


In [59]:
# Number of unique pixels

uniq_pix = df['PIX'].unique()
len(uniq_pix)

3369

## Searching for quasars

In [69]:
bit_boss = [10,11,12,13,14,15,16,17,18,19,40,41,42,43,44]
bit_eboss = [10,11,12,13,14,15,16,17,18]

def is_quasar(dataset, mask_bit):
    is_qso  = lambda b: ((df[dataset] & 2**b) > 0)
    all_qso = map(is_qso, mask_bit)
    return reduce(lambda x, y: x |  y , all_qso)

In [70]:
print 'total qso in BOSS_TARGET1 =  ', is_quasar('BOSS_TARGET1', bit_boss).sum()
print 'total qso in EBOSS_TARGET0 = ', is_quasar('EBOSS_TARGET0', bit_eboss).sum()
print 'total qso in EBOSS_TARGET1 = ', is_quasar('EBOSS_TARGET1', bit_eboss).sum() 

total qso in BOSS_TARGET1 =   466098
total qso in EBOSS_TARGET0 =  54108
total qso in EBOSS_TARGET1 =  201742


In [74]:
#Filter only quasars

only_qso = df[ is_quasar('BOSS_TARGET1', bit_boss)   |
                 is_quasar('EBOSS_TARGET0', bit_eboss) | 
                 is_quasar('EBOSS_TARGET1', bit_eboss)]
#only_qso.head()
len(only_qso)  

721948

## indexing and sorting by index


In [84]:
all_qso = only_qso.set_index(['PIX','THING_ID']).sort_index()

In [85]:
all_qso.head()

BOSS_TARGET1        DEC  EBOSS_TARGET0  EBOSS_TARGET1  \
PIX THING_ID                                                            
391 552894319  2199023796224  68.381197              0              0   
    552921545  2199023796224  68.418125              0              0   
392 552514273  3298535424000  68.011275              0              0   
    552514407  3298535424000  68.070896              0              0   
    552709867  2199023796224  68.289380              0              0   

               FIBERID    MJD  PLATE          RA  ZWARNING  
PIX THING_ID                                                
391 552894319      622  56667   7115  175.258222         0  
    552921545      624  56667   7115  175.582630         0  
392 552514273      932  56748   6966  185.714425         0  
    552514407      886  56748   6966  185.578268         0  
    552709867      880  56748   6966  185.466450         0

In [101]:
len(all_qso)

721948

### Select  a certain pixel info

In [145]:
#only the first rows of pixel 393
all_qso.query('PIX == 392 | PIX == 796')[:20]

BOSS_TARGET1        DEC  EBOSS_TARGET0  EBOSS_TARGET1  \
PIX THING_ID                                                            
392 552514273  3298535424000  68.011275              0              0   
    552514407  3298535424000  68.070896              0              0   
    552709867  2199023796224  68.289380              0              0   
    552709879  2199023271936  68.180993              0              0   
    552727940  3298535424000  68.328449              0              0   
    552898122  2199023796224  68.412267              0              0   
    552969567  2199023796224  68.459325              0              0   
    553069237  3298535407616  68.482644              0              0   
    553070107  3298535424000  68.467941              0              0   
    553213422  2199023255552  68.638159              0              0   
    553217575  3298535424000  68.629558              0              0   
    553237440  3298535424000  68.621416              0              0   
    553237790  3298534899712  68.665622              0              0   
    553238479  3298535424000  68.664666              0              0   
796 523898927              0  59.092429              0           1024   
    523898984              0  59.065369              0           2048   
    524093213              0  59.032789              0           2048   
    524093395              0  59.091369              0           2048   
    524269284              0  59.113129              0           2048   
    524269474              0  59.240716              0           3072   

               FIBERID    MJD  PLATE          RA  ZWARNING  
PIX THING_ID                                                
392 552514273      932  56748   6966  185.714425         0  
    552514407      886  56748   6966  185.578268         0  
    552709867      880  56748   6966  185.466450         0  
    552709879      882  56748   6966  185.445041         4  
    552727940      918  56748   6966  185.944277         0  
    552898122      826  56748   6966  184.380158         0  
    552969567      798  56748   6966  184.046478         0  
    553069237      912  56748   6966  185.698665         0  
    553070107      800  56748   6966  184.295837         0  
    553213422      748  56748   6966  183.883804         0  
    553217575      908  56748   6966  185.984029         0  
    553237440      866  56748   6966  185.483022         0  
    553237790      907  56748   6966  185.788208         0  
    553238479      832  56748   6966  184.373920         0  
796 523898927      957  57431   8185  162.841611         0  
    523898984      959  57431   8185  162.854731         0  
    524093213      519  57426   8184  162.947778         0  
    524093395      509  57426   8184  163.161187         0  
    524269284      950  57431   8185  163.247662         0  
    524269474      941  57431   8185  163.239711         0

In [94]:
print '# of THING_IDs within this pixel =', len(all_qso.loc[392].index)
print '# of unique THING_IDs within this pixel =', len(all_qso.loc[392].index.unique())

# of THING_IDs within this pixel = 14
# of unique THING_IDs within this pixel = 14


In [95]:
cols = ['THING_ID','Z']
ss = pd.DataFrame({col: fits[1][col].read().byteswap().newbyteorder() for col in cols})